In [1]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import statistics
import pickle

# Train & Test
# ==============================================================================
from sklearn.model_selection import train_test_split

# Proyección
# ==============================================================================
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn import metrics

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("datos/train_preproc_estandard_onehot_map.csv",index_col=0)

In [3]:
df.head()

,carat,x,y,z,price,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,-1.045515,-1.264806,-1.302517,-1.238169,6.353,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,0.447297,0.615030,0.653948,0.724458,9.183,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,-0.162443,-0.017521,0.007776,0.002904,7.983,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.594475,0.721940,0.689846,0.839906,8.371,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,-0.919362,-1.095531,-1.060202,-1.036134,6.588,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [4]:
df.columns

Index(['carat', 'x', 'y', 'z', 'price', 'cut_Fair', 'cut_Good', 'cut_Ideal',
       'cut_Premium', 'cut_Very Good', 'color_D', 'color_E', 'color_F',
       'color_G', 'color_H', 'color_I', 'color_J', 'clarity_I1', 'clarity_IF',
       'clarity_SI1', 'clarity_SI2', 'clarity_VS1', 'clarity_VS2',
       'clarity_VVS1', 'clarity_VVS2'],
      dtype='object')

In [5]:
# Reparto de datos en train y test
# ==============================================================================
# lo primero que hacemos es definir cual es nuestra variable Y y nuestras variables X
X = df.drop('price', axis =1)
y = df['price']


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size   = 0.8, random_state = 42)

In [6]:
df.head(1)

,carat,x,y,z,price,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,-1.045515,-1.264806,-1.302517,-1.238169,6.353,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [7]:
print("Datos de entrenamiento")
print("-----------------------")
print(y_train.describe())
print("Datos de testeo")
print("-----------------------")
print(y_test.describe())

Datos de entrenamiento
-----------------------
count    32348.000000
mean         7.786322
std          1.015895
min          5.814000
25%          6.855000
50%          7.782000
75%          8.577000
max          9.842000
Name: price, dtype: float64
Datos de testeo
-----------------------
count    8087.000000
mean        7.771408
std         1.019746
min         5.787000
25%         6.837500
50%         7.767000
75%         8.590500
max         9.842000
Name: price, dtype: float64


In [8]:
# create a regressor object
regressor = DecisionTreeRegressor(random_state = 0) 
  
# fit the regressor with X and Y data
regressor.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

In [9]:
# max features

max_features = np.sqrt(len(X_train.columns))
max_features

4.898979485566356

In [10]:
# max depth

max_depth = regressor.tree_.max_depth
max_depth

35

In [11]:
# hacemos las predicciones sobre los dos set de datos el X_test y el X_train
y_pred_test_dt = regressor.predict(X_test)
y_pred_train_dt = regressor.predict(X_train)

In [12]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [13]:
# sacamos las métricas para ver si hay overfitting

dt_results1 = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "Decission Tree I")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,0.087501,0.016719,0.129302,0.983920,test,Decission Tree I
1,0.000372,0.000030,0.005511,0.999971,train,Decission Tree I


In [15]:
# Generación del GridSearch
# ==============================================================================

# definimos un diccionario con los hiperparámetros que queremos testear. 
param = {"max_depth": [4,6,8,10,12,14,16,18],
        "min_samples_split": [500,600,700,800],
        "min_samples_leaf": [500,600,700,800],
        "max_features": [1,2,3,4,5]}

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = GradientBoostingRegressor(),
        param_grid = param,
        n_jobs     = -1,
        verbose    = 1,
        cv         = 10, 
        return_train_score = True
       )

grid.fit(X = X_train, y = y_train)

Fitting 10 folds for each of 640 candidates, totalling 6400 fits


GridSearchCV(cv=10, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'max_depth': [4, 6, 8, 10, 12, 14, 16, 18],
                         'max_features': [1, 2, 3, 4, 5],
                         'min_samples_leaf': [500, 600, 700, 800],
                         'min_samples_split': [500, 600, 700, 800]},
             return_train_score=True, verbose=1)

In [17]:
# Mejores hiperparámetros por validación cruzada
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv)")
print("----------------------------------------")
print(grid.best_params_)

----------------------------------------
Mejores hiperparámetros encontrados (cv)
----------------------------------------
{'max_depth': 14, 'max_features': 5, 'min_samples_leaf': 500, 'min_samples_split': 700}


In [18]:
y_pred_gb_test= grid.predict(X_test)
y_pred_gb_train= grid.predict(X_train)

In [20]:
dt_results2 = metricas(y_test, y_train, y_pred_gb_test, y_pred_gb_train, "Gradient Boosting")
dt_results2

,MAE,MSE,RMSE,R2,set,modelo
0,0.086191,0.014576,0.120731,0.985981,test,Gradient Boosting
1,0.083752,0.013684,0.116979,0.986740,train,Gradient Boosting


In [22]:
with open(f'datos/gradientboosting.pkl', 'wb') as gradientboosting1:
        pickle.dump(grid, gradientboosting1)

In [23]:
df.columns

Index(['carat', 'x', 'y', 'z', 'price', 'cut_Fair', 'cut_Good', 'cut_Ideal',
       'cut_Premium', 'cut_Very Good', 'color_D', 'color_E', 'color_F',
       'color_G', 'color_H', 'color_I', 'color_J', 'clarity_I1', 'clarity_IF',
       'clarity_SI1', 'clarity_SI2', 'clarity_VS1', 'clarity_VS2',
       'clarity_VVS1', 'clarity_VVS2'],
      dtype='object')